In [71]:
import numpy as np
import sklearn.svm as svm
from sklearn.externals import joblib
from skimage.feature import hog

import random
import glob
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import gluoncv
import gym_project.envs.object_localization as object_localization

In [72]:
def trainSvm(datas, labels):
    clf = svm.SVC(kernel='linear', C = 0.6 , gamma = 20, decision_function_shape='ovo')
    print ("Training a SVM")
    clf.fit(datas, labels)
    score = clf.score(datas, labels)
    print (score)
    return clf,score

use the idea of sliding windiws from https://www.zhihu.com/question/46292829

In [64]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, 224, stepSize):
        for x in range(0, 224, stepSize):   
              yield(x, y, image[y:y+windowSize[1], x:x+windowSize[0]])

In [78]:
normalize = True
visualize = False
block_norm = 'L2-Hys'
cells_per_block = [2,2]
pixels_per_cell = [5,5]
orientations = 9
new_model_path = './models/svm'
HOG = []
result = [] 
negative_img_path = 'C:/ZZF/SVM/VOCtrainval_11-May-2012/VOCdevkit'

clf = ssv.SVC(kernel='linear', C = 0.6 , gamma = 20, decision_function_shape='ovo')

voc_dataset = gluoncv.data.VOCDetection(root=negative_img_path, splits=[(2012, 'trainval')])

for childDir in os.listdir(voc_dataset):
    f = os.path.join(negative_img_path, childDir)
    data = cv2.imread(f)
    #plt.imshow(f)
    scales = [(100, 100), (200,200)]

    for winW,winH in scales:
        for (x, y, window) in sliding_window(data, stepSize=10, windowSize=(winW,winH)):
            result = 0    
            window = cv2.resize(window,(224,224),interpolation=cv2.INTER_CUBIC) 
            gray = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
            window_fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm, visualize, normalize)
            print(window_fd)
            result = int(clf.predict(window_fd))

            if result >= 0.5:
                HOG.append(window_fd)
                result.append(result)

New_clf = trainSvm(HOG, result)
joblib.dump(new_clf, new_model_path)
print ("Classifier saved to {}".format(new_model_path))

TypeError: listdir: path should be string, bytes, os.PathLike or None, not VOCDetection